In [548]:
import xgboost as xgb
import lightgbm as lgb
import pandas as pd
import numpy as np
from datetime import date
import datetime
import warnings
warnings.filterwarnings("ignore")

In [549]:
import gc

In [550]:
gc.collect()

461

In [551]:
flow_train = pd.read_csv('../data/flow_train.csv')
tran_train = pd.read_csv('../data/transition_train.csv')

# =========数据初探===============

In [552]:
print(flow_train.shape,tran_train.shape)
#flow_train,6列数据，2.7万样本，1个日期，2文本，3数据列，3M多内存
#tran_train,6列数据，24.8万样本，1日期，4文本，1数据列，382M多内存

(26852, 6) (2480320, 6)


In [553]:
print(flow_train.city_code.value_counts())

c7537db4101856877ea6381d0174283c    6028
3f7f0ce35d6d0a08377eb2efe2189f4f    6028
58a33c947775af5de36841c9f553317d    4384
a20d041605db832309e26c003c626719    4384
06d86ef037e4bd311b94467c3320ff38    3014
5615dc7c1af1f7dabd80bd8b8ecb1ea0    2740
ee2ff207184bf16b4a0aec0f97900c27     274
Name: city_code, dtype: int64


In [554]:
# print(flow_train.columns)
# #日期，城市编码，区县编码，当天驻留人数，当天流入人数，当天流出人数
# print(tran_train.columns)
# #日期，出发城市编号，出发区县编号，到达城市编号，到达区县编号

In [555]:
# print(tran_train.date_dt.min())
# print(tran_train.date_dt.max())
# print(tran_train.year.value_counts())
# print(tran_train.month.value_counts())
# print(tran_train.day.value_counts())

#flow_train 2017.6.1-2018.3.1 
#tran_train 2017.6.1-2018.3.1
#2017.6.1为周四

#所以未来15天的预测 ，便是从2018.3.2-2018.3.16
#所以分别预测驻留人数、流入人数、流出人数，那么分别将他们依次作为标签去进行预测
#是否可以即把他们当作标签又当作特征，当预测dwell时候，flow_in,flow_out均作为特征，
#然后预测flow_in时，将dwell,flow_out作为特征
#是否可以将（flow_in,flow_out）作为一个特征呢？？？不妨尝试下！

In [556]:
#以下为算法乱想，忽略
#flow_in和flow_out之间满足线性关系，y=a(flow_in)+b(flow_out)？
#而a b分别通过以上的特征进行预测，又是一个线性关系，例a=w1*x1+w2*x2+w3*x3,b也是如此
#所以变成一个算法预测a b，然后得到新的特征a b，再用linearRegressor去预测y

In [557]:
flow_train['flow_in_dwell']=round(flow_train['flow_in']/flow_train['dwell'],6)
flow_train['flow_out_dwell']=round(flow_train['flow_out']/flow_train['dwell'],6)
flow_train['flow_in_out']=round(flow_train['flow_in']/flow_train['flow_out'],6)

In [558]:
gc.collect()

28

In [559]:
flow_train['address'] = flow_train['city_code']+':'+flow_train['district_code']
address = list(set(flow_train['address']))

tran_train['start_address']=tran_train['o_city_code']+':'+tran_train['o_district_code']
tran_train['arr_address']=tran_train['d_city_code']+':'+tran_train['d_district_code']
start_address=list(set(tran_train['start_address']))
arr_address=list(set(tran_train['arr_address']))

#### <font color=red>此处代码证明flow_train的数据来自于tran_train，所以大可以安心的从tran_train中提取新特征即可<font>

In [560]:
#根据tran_train计算98个区县的流出人数=sum(cnt)
# tran_train['address']=tran_train['start_address']
# tmp=tran_train.groupby(['date_dt','address'],as_index=False)['cnt'].agg({'cnt':'sum'})
# tmp.reset_index()
# tmp.rename(columns={'cnt':'flow_out_cnt'},inplace=True)
# # print(tmp)
# flow_train=pd.merge(flow_train,tmp,on=['date_dt','address'],how='left')
# # print(flow_train.shape)

# #事实证明，flow_out_cnt==flow_out,因此flow_train.address==tran_train.start_address

In [561]:
# tt=tran_train.groupby(['date_dt','arr_address'],as_index=False)['cnt'].agg({'cnt':'sum'})
# tt.reset_index()
# tt.rename(columns={'cnt':'flow_in_cnt'},inplace=True)

# tt['address']=tt['arr_address']

# flow_train=pd.merge(flow_train,tt,on=['date_dt','address'],how='left')
# # print(flow_train[['flow_in','flow_in_cnt']])

# 事实证明，flow_in_cnt==flow_in
# 因此，flow_train中的flow_in and flow_out均来自于tran_train表格，并且没有遗漏，数据完全吻合
# 因此，只要从tran_train中提取feature，对应的拼接到flow_train中即可

#### <font color=red>以上代码证明flow_train的数据来自于tran_train，所以大可以安心的从tran_train中提取新特征即可<font>

# ===对tran_train处理,得到新特征合并到flow_train里=====

In [562]:
print(flow_train.shape)

(26852, 10)


In [563]:
tmp=tran_train.groupby(['date_dt','start_address'],as_index=False)['cnt'].agg({'cnt':'count'})
tmp['address']=tmp['start_address']
tmp['flow_out_count']=tmp['cnt']
flow_train=flow_train.merge(tmp[['date_dt','address','flow_out_count']],how='left',on=['date_dt','address'])

In [564]:
del tmp

In [565]:
tmp=tran_train.groupby(['date_dt','arr_address'],as_index=False)['cnt'].agg({'cnt':'count'})
tmp['address']=tmp['arr_address']
tmp['flow_in_count']=tmp['cnt']
flow_train=flow_train.merge(tmp[['date_dt','address','flow_in_count']],how='left',on=['date_dt','address'])
del tmp
gc.collect()

128

In [566]:
print(flow_train.columns)

Index(['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in', 'flow_out',
       'flow_in_dwell', 'flow_out_dwell', 'flow_in_out', 'address',
       'flow_out_count', 'flow_in_count'],
      dtype='object')


In [567]:
def year(date):
    date = str(date)
    return int(date[0:4])
def month(date):
    date = str(date)
    return int(date[4:6])
def day(date):
    date = str(date)
    return int(date[6:8])

flow_train['year'] = flow_train['date_dt'].apply(year)
flow_train['month'] = flow_train['date_dt'].apply(month)
flow_train['day'] = flow_train['date_dt'].apply(day)

In [568]:
from datetime import datetime

flow_train['weekd']=flow_train['date_dt'].apply(lambda x:datetime.strptime(str(x),'%Y%m%d').weekday())
flow_train['weekd']=flow_train['weekd']+1
flow_train['weekd']=flow_train['weekd'].astype(int)

#1-7分别代表周一到周日
# tran_train['weekd']=tran_train['date_dt'].apply(lambda x:datetime.strptime(str(x),'%Y%m%d').weekday())
# tran_train['weekd']=tran_train['weekd']+1

#分成上中下旬，不过用weekd的特征已经代替了，再加上便是冗余数据了
# def get_xun(x):
#     if x<=10:
#         return 1
#     elif x<=20:
#         return 2
#     else:
#         return 3
    
# flow_train['xun']=0
# flow_train['xun']=flow_train['day'].apply(get_xun)
# tran_train['xun']=0
# tran_train['xun']=tran_train['xun'].apply(get_xun)

In [569]:
print(flow_train.columns)

Index(['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in', 'flow_out',
       'flow_in_dwell', 'flow_out_dwell', 'flow_in_out', 'address',
       'flow_out_count', 'flow_in_count', 'year', 'month', 'day', 'weekd'],
      dtype='object')


In [570]:
flow_train['flow_in_mean']=round(flow_train['flow_in']/flow_train['flow_in_count'],6)
flow_train['flow_out_mean']=round(flow_train['flow_out']/flow_train['flow_out_count'],6)

In [571]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
flow_train['address_le']=le.fit_transform(flow_train['address'])

#### 1.区县之间的距离？？？<br><br>2.一维的交叉特征的nunique\count\sum\mean\std\crt基本特征，由于还不是特别数量，建议一个一写！<br><br>3.同一个区县的样本进出数量，日平均进出样本数量；星期1234567分别进出样本数量；<br><br>4.离开某一区县的第一次时间，对应的是星期几，第二次时间，对应的是星期几，对应的间隔时间，看看是否有规律可寻；以及进入某一区县的间隔天数，离开某一区县进入 另一区县的间隔天数，每月的次数合计，17年合计，18年合计，17年平均，18年平均；

# ===============以下模型部分==================

可以确定是回归模型：
每一群模型，建立一个Pipeline去执行察看结果
#模型：LinearRegressor
#模型：Lasso\Ridge加入了l1 l2正则化，防止过拟合
#模型：神经网络LasticNet？

#模型：SVM
#模型：KNN（按照城市各自训练的时候，感觉比较适用）

#模型：GDBT
#模型：lightGBM训练，但是容易过拟合，需要好好优化参数
#模型：XGBoost
#模型：Adaboost
#模型：Catboost

#模型：以上最好的模型用BaggingRegressor()融合
#模型：以上几个，bagging(选择几个比较好的模型，按照成绩进行权重分配)
#模型：以上几个，blending(选择好的与不好的几个，进行blending)
#模型：基本属于线性可分型，不适用stacking融合#

In [572]:
def rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

In [573]:
import datetime

In [574]:
star = '20180301'
dates = []
for i in range(1,16):
    date_format = datetime.datetime.strptime(star,'%Y%m%d')
    fut_date = date_format + datetime.timedelta(days=i)
    dates.append(int(datetime.datetime.strftime(fut_date,'%Y%m%d')))
# print(dates)

In [575]:
test_df = pd.DataFrame({'date_dt':dates})
test_df['year'] = test_df['date_dt'].apply(year)
test_df['month'] = test_df['date_dt'].apply(month)
test_df['day'] = test_df['date_dt'].apply(day)
test_df['date_dt'] = test_df['date_dt'].astype(str)

In [576]:
result = pd.DataFrame(columns=['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in', 'flow_out'])

In [577]:
flow_train_1 = flow_train[flow_train['date_dt'] < 20180214]
flow_test_1 = flow_train[flow_train['date_dt'] >= 20180214]

In [580]:
label = ['dwell','flow_in','flow_out']
feature = ['flow_in_dwell','flow_out_dwell','flow_in_out','flow_out_count','flow_in_count','year',
          'month','day','weekd','flow_in_mean','flow_out_mean','address_le']

In [581]:
for ad in address:
    ad_split = ad.split(':')
    test_df['city_code'] = ad_split[0]
    test_df['district_code'] = ad_split[1]
    
    for y in label:
        train_x = flow_train[flow_train['address']==ad][feature]
        train_y = flow_train[flow_train['address']==ad][y]
        test_x = flow_test_1[flow_test_1['address']==ad][feature]
        test_y = flow_test_1[flow_test_1['address']==ad][y]
        
        gbm = lgb.LGBMRegressor(num_leaves=50,
                        learning_rate=0.05,
                        n_estimators=1000)
        
        gbm.fit(train_x, train_y, 
                eval_set=[(test_x, test_y)],
                eval_metric='l1',
                early_stopping_rounds=50)
        
        # predict
        y_pred = gbm.predict(test_x, num_iteration=gbm.best_iteration_)
        # eval
        print('The rmsle of prediction is:', rmsle(test_y, y_pred)[1])
        
        test_df[y] = gbm.predict(test_df[feature])
        
    result = pd.concat([result,test_df[result.columns]])

[1]	valid_0's l1: 25.7687
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l1: 24.5175
[3]	valid_0's l1: 23.2233
[4]	valid_0's l1: 21.9311
[5]	valid_0's l1: 20.8786
[6]	valid_0's l1: 19.7658
[7]	valid_0's l1: 18.6619
[8]	valid_0's l1: 17.6277
[9]	valid_0's l1: 16.6407
[10]	valid_0's l1: 15.7905
[11]	valid_0's l1: 14.9251
[12]	valid_0's l1: 14.1581
[13]	valid_0's l1: 13.3926
[14]	valid_0's l1: 12.6496
[15]	valid_0's l1: 12.0287
[16]	valid_0's l1: 11.4129
[17]	valid_0's l1: 10.7698
[18]	valid_0's l1: 10.2209
[19]	valid_0's l1: 9.68541
[20]	valid_0's l1: 9.10159
[21]	valid_0's l1: 8.52576
[22]	valid_0's l1: 8.07405
[23]	valid_0's l1: 7.63391
[24]	valid_0's l1: 7.28163
[25]	valid_0's l1: 6.82898
[26]	valid_0's l1: 6.48747
[27]	valid_0's l1: 6.12216
[28]	valid_0's l1: 5.81819
[29]	valid_0's l1: 5.37889
[30]	valid_0's l1: 5.16695
[31]	valid_0's l1: 4.93243
[32]	valid_0's l1: 4.66791
[33]	valid_0's l1: 4.31503
[34]	valid_0's l1: 4.07617
[35]	valid_0's l1: 3.85216
[3

KeyError: "['flow_in_dwell' 'flow_out_dwell' 'flow_in_out' 'flow_out_count'\n 'flow_in_count' 'weekd' 'flow_in_mean' 'flow_out_mean' 'address_le'] not in index"

In [ ]:
result['date_dt'] = result['date_dt'].astype(int)
result = result[['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in', 'flow_out']]
result.to_csv('..\data\predictionV1.csv', index=False, header=None)